In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [18]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json
import numpy as np

In [11]:
from probreg import transformation as tf
from probreg import cpd
from probreg import callbacks
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [3]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi

from exact_sync.v1.models import ImageSet, Product, AnnotationType, Image, Annotation
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [4]:
configuration = Configuration()
configuration.username = '#####' #'exact'
configuration.password = '####' #'exact'
configuration.host = "https://euroexact01.euroimmun.de/" #"http://127.0.0.1:8000"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

In [54]:
annotation_types_points = {}          
annotation_type_target = None

for product in product_api.list_products(name="ColonCancer").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id, name="LymphaticVessel").results:
        annotation_type_target = annotation_type
    
for product in product_api.list_products(name="Reference_Points").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id).results:
        annotation_types_points[annotation_type.id] = annotation_type

        
        
annotation_types_points

{11: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FFA500',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 11,
  'name': 'L0',
  'node_count': 0,
  'product': 4,
  'sort_order': 2,
  'vector_type': 1},
 12: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FFA500',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 12,
  'name': 'L1',
  'node_count': 0,
  'product': 4,
  'sort_order': 3,
  'vector_type': 1},
 13: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FFA500',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 13,
  'name': 'L2',
  'node_count': 0,
  'product': 4,
  'sort_order': 4,
  'vector_type': 1},
 14: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FFA500',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_co

In [55]:
annotation_type_target

{'area_hit_test': True,
 'closed': True,
 'color_code': '#FF0000',
 'default_height': 500,
 'default_width': 500,
 'enable_blurred': False,
 'enable_concealed': False,
 'id': 10,
 'name': 'LymphaticVessel',
 'node_count': 0,
 'product': 2,
 'sort_order': 0,
 'vector_type': 1}

In [14]:
names = [("CRC-A1-5 40X.tif", 5, "HE"), ("CRC-A1-1 40X.tif", 1, "HE"), 
         ("CRC-A1-5_stitched.tif", 5, "D2-40"), ("CRC-A1-1_2_stitched.tif", 1, "D2-40")]

In [41]:
annotations = []

for name, pair_id, stain  in names:
    
    image = images_api.list_images(name=name).results[0]
    
    for anno in annotations_api.list_annotations(image=image.id, deleted=False, pagination=False,
                                                 fields="annotation_type,id,image,vector,unique_identifier",
                                                 ).results:
        
        if anno.annotation_type in annotation_types_points:
            annotations.append([image.id, image.name, stain, pair_id, anno.id, anno.vector, anno.unique_identifier, annotation_types_points[anno.annotation_type].name])
        
annotations = pd.DataFrame(annotations, columns=["image_id", "image_name", "stain", "pair", "id", "vector", "unique_identifier", "annotation_type"])


annotations["x1"] = [vector["x1"] for vector in annotations["vector"]]
annotations["x2"] = [vector["x2"] for vector in annotations["vector"]]
annotations["y1"] = [vector["y1"] for vector in annotations["vector"]]
annotations["y2"] = [vector["y2"] for vector in annotations["vector"]]


annotations["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(annotations["x1"], annotations["x2"])]
annotations["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(annotations["y1"], annotations["y2"])]

annotations.head()

,image_id,image_name,stain,pair,id,vector,unique_identifier,annotation_type,x1,x2,y1,y2,center_x,center_y
0,20,CRC-A1-5 40X.tif,HE,5,30844,"{'x1': 9378, 'x2': 9578, 'y1': 15363, 'y2': 15...",a8c564e8-fd69-43b7-b9a5-f1f16c7bbbd5,L0,9378,9578,15363,15563,9478.0,15463.0
1,20,CRC-A1-5 40X.tif,HE,5,30845,"{'x1': 8299, 'x2': 8499, 'y1': 36048, 'y2': 36...",9a8e6a0c-0a43-4f6c-b6d5-4eeeca8b0dcc,L1,8299,8499,36048,36248,8399.0,36148.0
2,20,CRC-A1-5 40X.tif,HE,5,30846,"{'x1': 7603, 'x2': 7803, 'y1': 54247, 'y2': 54...",aacda99c-5297-402f-bddc-8d043aeff4ed,L2,7603,7803,54247,54447,7703.0,54347.0
3,20,CRC-A1-5 40X.tif,HE,5,30847,"{'x1': 8638, 'x2': 8838, 'y1': 83095, 'y2': 83...",74a268f2-dab0-497f-b11b-57dd327313ba,L3,8638,8838,83095,83295,8738.0,83195.0
4,20,CRC-A1-5 40X.tif,HE,5,30848,"{'x1': 2225, 'x2': 2425, 'y1': 96279, 'y2': 96...",88a2ff08-15a3-40e9-9005-3ded346f4340,L4,2225,2425,96279,96479,2325.0,96379.0


In [47]:
source_anno["image_name"].unique()[0]

'CRC-A1-1_2_stitched.tif'

In [44]:
tf_param.b

array([[ 0.55464862,  0.0092813 ],
       [-0.01803478,  0.5429615 ]])

In [45]:
tf_param.t

array([-12090.45440953,   1028.76986763])

In [49]:
%%time

results = {}

for pair in annotations["pair"].unique():
    
    source_anno = annotations[(annotations["pair"] == pair) & (annotations["stain"] == "D2-40")]
    target_anno = annotations[(annotations["pair"] == pair) & (annotations["stain"] == "HE")]
    
    source =  np.array([[center_x, center_y] for center_x, center_y in zip(source_anno["center_x"], source_anno["center_y"])])
    target =  np.array([[center_x, center_y] for center_x, center_y in zip(target_anno["center_x"], target_anno["center_y"])])
    
    tf_param, sigma2, q = cpd.registration_cpd(source, target, 'affine') # ‘rigid’, ‘affine’, ‘nonrigid’   
    
    
    results[source_anno["image_name"].unique()[0]] = {
        "target": target_anno["image_name"].unique()[0],
        "b": tf_param.b,
        "t": tf_param.t,
        "sigma2": sigma2,
        "q": q
    }

Wall time: 46 ms


In [50]:
results

{'CRC-A1-5_stitched.tif': {'target': 'CRC-A1-5 40X.tif',
  'b': array([[ 0.55519829, -0.05122972],
         [ 0.05300909,  0.53146395]]),
  't': array([-3687.08431348,  1405.46596613]),
  'sigma2': 510090.1898897553,
  'q': 353.5585707996683},
 'CRC-A1-1_2_stitched.tif': {'target': 'CRC-A1-1 40X.tif',
  'b': array([[ 0.55464862,  0.0092813 ],
         [-0.01803478,  0.5429615 ]]),
  't': array([-12090.45440953,   1028.76986763]),
  'sigma2': 249929.90756729126,
  'q': 335.7233946950652}}

In [63]:
annotations_to_upload = []
urls = []


for source_name, result in results.items():
    
    target_name = result["target"]
    tf_param = tf.AffineTransformation(result["b"], result["t"])
    
    source_image = images_api.list_images(name=source_name).results[0]
    taret_image = images_api.list_images(name=target_name).results[0]
    
    for anno in annotations_api.list_annotations(image=source_image.id, deleted=False, pagination=False, 
                                                 annotation_type=annotation_type_target.id,
                                                 fields="annotation_type,id,image,vector,unique_identifier",
                                                 ).results:
        
        xmin, ymin, xmax, ymax = anno.vector["x1"], anno.vector["y1"], anno.vector["x2"], anno.vector["y2"]
        source_url = f"https://euroexact01.euroimmun.de/annotations/{source_image.id}/?xmin={xmin}&ymin={ymin}&xmax={xmax}&ymax={ymax}"

        x1, y1 = tf_param.transform([xmin, ymin])
        x2, y2 = tf_param.transform([xmax, ymax])
        vector = {"x1": x1, "y1": y1, "x2": x2, "y2": y2}
        
        target_url = f"https://euroexact01.euroimmun.de/annotations/{taret_image.id}/?xmin={x1}&ymin={y1}&xmax={x2}&ymax={y2}"
    
        urls.append((source_url, target_url))
    
        anno = Annotation(annotation_type=anno.annotation_type, 
                          vector=vector, 
                          unique_identifier=anno.unique_identifier,
                          image=taret_image.id)

        annotations_to_upload.append(anno)    

In [64]:
for source_url, target_url in urls[:5]:
    print(f"{source_url} ->  {target_url}") 

https://euroexact01.euroimmun.de/annotations/10/?xmin=151583&ymin=111317&xmax=153538&ymax=112283 ->  https://euroexact01.euroimmun.de/annotations/20/?xmin=74768.79862207935&ymin=68601.71477155056&xmax=75804.7233613096&ymax=69218.74171205441
https://euroexact01.euroimmun.de/annotations/10/?xmin=151483&ymin=112006&xmax=151900&ymax=112261 ->  https://euroexact01.euroimmun.de/annotations/20/?xmin=74677.98151478675&ymin=68962.59252178599&xmax=74896.43562125953&ymax=69120.22061808076
https://euroexact01.euroimmun.de/annotations/10/?xmin=152122&ymin=112272&xmax=152372&ymax=112522 ->  https://euroexact01.euroimmun.de/annotations/20/?xmin=75019.12611400655&ymin=69137.83473916422&xmax=75145.11825517846&ymax=69283.95299792799
https://euroexact01.euroimmun.de/annotations/10/?xmin=150545&ymin=106389&xmax=153694&ymax=108861 ->  https://euroexact01.euroimmun.de/annotations/20/?xmin=74444.96287133126&ymin=65927.63700951266&xmax=76066.64240346897&ymax=67408.34150516157
https://euroexact01.euroimmun.de/

In [65]:
annotations_to_upload[:3]

[{'annotation_type': 10,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 20,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': 'e5820fbb-7971-461d-ba5c-d69ceb502a04',
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 74768.79862207935,
             'x2': 75804.7233613096,
             'y1': 68601.71477155056,
             'y2': 69218.74171205441},
  'verified_by_user': None},
 {'annotation_type': 10,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 20,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': '162df800-118b-488f-892e-95727f77979b',
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 74677.98151478675,
             'x2': 74896.43562125953,
             'y1': 

In [66]:
#annotations_api.create_annotation(body=annotations_to_upload)

[{'annotation_type': 10,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 30944,
  'image': 20,
  'last_edit_time': datetime.datetime(2021, 1, 16, 13, 49, 21, 991814),
  'last_editor': 2,
  'meta_data': None,
  'time': datetime.datetime(2021, 1, 16, 13, 49, 21, 991793),
  'unique_identifier': 'e5820fbb-7971-461d-ba5c-d69ceb502a04',
  'uploaded_media_files': [],
  'user': 2,
  'vector': {'x1': 74768.79862207935,
             'x2': 75804.7233613096,
             'y1': 68601.71477155056,
             'y2': 69218.74171205441},
  'verified_by_user': 'False'},
 {'annotation_type': 10,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 30945,
  'image': 20,
  'last_edit_time': datetime.datetime(2021, 1, 16, 13, 49, 21, 991880),
  'last_editor': 2,
  'meta_data': None,
  'time': datetime.datetime(2021, 1, 16, 13, 49, 21, 991870),
  'unique_iden